In [1]:
from PIL import Image 
import math

In [2]:
d = 1
Vw = 1
Vh = 1

In [3]:
center = [ (0, 31, 30), (1.5, 0, 2), (-1.5, 0, 2), (0, 3, -5)]
light_position = (0, 3, -5)
clr = [ (242, 221, 222), (0, 120, 250), (20, 210, 255), (0, 0, 0)]
radius = [ 40, 1, 1, 0.01]
specular = [ 40, 40, 40, 0]
intensity = (0.1, 0.9)
reflective = (0.2, 0.6, 0.4, 0)
t_max = 1

In [4]:
def CanvasToViewport(x, y): 
    return (x*Vw/Cw, y*Vh/Ch, d)

In [5]:
def length(V):
    return math.sqrt(V[0]*V[0] + V[1]*V[1] + V[2]*V[2])

In [6]:
def dot(V1, V2):
    return V1[0]*V2[0] + V1[1]*V2[1] + V1[2]*V2[2]

In [7]:
def neg(V):
    return (-V[0], -V[1], -V[2])

In [8]:
def sumv(V1, V2):
    return (V1[0]+V2[0], V1[1]+V2[1], V1[2]+V2[2])

In [9]:
def ReflectRay(R, N):
    X = (2*N[0]*dot(N,R)-R[0], 2*N[1]*dot(N,R)-R[1], 2*N[2]*dot(N,R)-R[2])
    return X

In [10]:
def ComputeLighting(P, N, V, s):
    i = 0.0
    for num in range(0, 2):
        if (num == 0):
            i += intensity[0]
        else:    
            L = (light_position[0] - P[0], 
                 light_position[1] - P[1], 
                 light_position[2] - P[2])
            
            a1 = 0
            a2 = 1
            
            v = []
            
            kol = 4
            
            for vec in range(0, 1):
                tmp = (light_position[0] - P[0]+a1, light_position[1] - P[1]+a2, light_position[2] - P[2])
                v.append(tmp)
                a1 += 4/kol
                a2 -= 4/kol
            for vec in range(1, 2):
                tmp = (light_position[0] - P[0]+a1, light_position[1] - P[1]+a2, light_position[2] - P[2])
                v.append(tmp)
                a1 -= 4/kol
                a2 -= 4/kol
            for vec in range(2, 3):
                tmp = (light_position[0] - P[0]+a1, light_position[1] - P[1]+a2, light_position[2] - P[2])
                v.append(tmp)
                a1 -= 4/kol
                a2 += 4/kol
            for vec in range(3, 4):
                tmp = (light_position[0] - P[0]+a1, light_position[1] - P[1]+a2, light_position[2] - P[2])
                v.append(tmp)
                a1 += 4/kol
                a2 += 4/kol    
                
            for k in range(0, kol):
                shadow_sphere = ClosestIntersection(P, v[k], 0.001, t_max)[0]
                shadow_t = ClosestIntersection(P, v[k], 0.001, t_max)[1]
            
                if (shadow_sphere != -1):
                    continue
                intens = intensity[1] / 4
                n_dot_l = dot(N, L)
        
                if (n_dot_l > 0):
                    i += intens * n_dot_l / (length(N)*length(L)) 
    
                if (s != -1): 
                    R = (2*N[0]*n_dot_l - L[0], 2*N[1]*n_dot_l - L[1], 2*N[2]*n_dot_l - L[2])
                    r_dot_v = dot(R, V)
                    if (r_dot_v > 0):
                        i += intens*pow(r_dot_v/(length(R)*length(V)), s)
    return i

In [11]:
def IntersectRaySphere(O, D, i):
    C = center[i]
    r = radius[i]
    
    OC = (O[0] - C[0], O[1] - C[1], O[2] - C[2])

    k1 = dot(D, D)
    k2 = 2 * dot(OC, D)
    k3 = dot(OC, OC) - r*r

    discriminant = k2*k2 - 4*k1*k3
    if (discriminant < 0):
        return (math.inf, math.inf) 

    t1 = (-k2 + math.sqrt(discriminant)) / (2*k1)
    t2 = (-k2 - math.sqrt(discriminant)) / (2*k1)
    return (t1, t2)

In [12]:
def ClosestIntersection(O, D, t_min, t_max):
    closest_t = 1000000
    closest_sphere = -1
    for k in range(0, 4):
        t1 = IntersectRaySphere(O, D, k)[0]
        t2 = IntersectRaySphere(O, D, k)[1]
    
        if (t1 > t_min and t1 < t_max and t1 < closest_t):
            closest_t = t1
            closest_sphere = k
        
        if (t2 > t_min and t2 < t_max and t2 < closest_t):
            closest_t = t2
            closest_sphere = k 
            
    return (closest_sphere, closest_t)

In [13]:
def TraceRay(O, D, t_min, t_max, depth): 
    
    closest_sphere = ClosestIntersection(O, D, t_min, t_max)[0]
    closest_t = ClosestIntersection(O, D, t_min, t_max)[1]
    
    if (closest_sphere == -1):
        return (50, 50, 50) 
    
    P = (O[0] + closest_t*D[0], O[1] + closest_t*D[1], O[2] + closest_t*D[2])   
    X = (P[0] - center[closest_sphere][0], 
         P[1] - center[closest_sphere][1], 
         P[2] - center[closest_sphere][2] )
    lenN = length(X)
    N = (X[0] / lenN, X[1] / lenN, X[2] / lenN) 
    
    X = (-D[0], -D[1], -D[2])
    
    local_color = (clr[closest_sphere][0]*ComputeLighting(P, N, X, specular[closest_sphere]), 
                   clr[closest_sphere][1]*ComputeLighting(P, N, X, specular[closest_sphere]), 
                   clr[closest_sphere][2]*ComputeLighting(P, N, X, specular[closest_sphere]) )
    r = reflective[closest_sphere]
    if (depth <= 0 or r <= 0):
        return local_color
    
    R = ReflectRay(X, N)
    reflected_color = TraceRay(P, R, 0.001, math.inf, depth - 1)

    res = (local_color[0]*(1 - r) + reflected_color[0]*r,
           local_color[1]*(1 - r) + reflected_color[1]*r,
           local_color[2]*(1 - r) + reflected_color[2]*r)
    return res

In [14]:
image = Image.open(r'canvas.png')  
  
Cw, Ch = image.size 
O = (0, 0, -15)  
    
for x in range(-300, 300):
    x1 = x + 300
    for y in range(-300, 300):
        y1 = y + 300
        color = TraceRay(O, CanvasToViewport(x, y), 0, math.inf, 1)
        image.putpixel((x1, y1), tuple(int(c) for c in color) )  
image.show() 

KeyboardInterrupt: 